In [39]:
import pandas as pd
import seaborn as sns
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
df = pd.read_csv('../seeds.txt')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 8 columns):
area                210 non-null float64
perimeter           210 non-null float64
compactness         210 non-null float64
kernel_length       210 non-null float64
kernel_width        210 non-null float64
asymmetry_coef      210 non-null float64
length_of_groove    210 non-null float64
targetClass         210 non-null int64
dtypes: float64(7), int64(1)
memory usage: 13.2 KB


In [41]:
X, y = df.drop('targetClass', axis=1), df.targetClass

est = RandomForestClassifier(n_jobs=-1, n_estimators=50)
grid = dict(max_depth=[2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, None],
            max_features=[1, 2, 3, 4, 5, 6, 7],
            min_samples_split=[2, 3, 4, 5, 6, 7, 8, 9, 10]
           )
gscv = GridSearchCV(est, param_grid=grid, cv=5)

In [42]:
gscv.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, None], 'max_features': [1, 2, 3, 4, 5, 6, 7], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [43]:
gscv.best_params_, gscv.best_score_

({'max_depth': 3, 'max_features': 6, 'min_samples_split': 6},
 0.92380952380952386)

In [45]:
rf = RandomForestClassifier(n_jobs=-1, n_estimators=100, **gscv.best_params_)
scores = cross_val_score(rf, X, y, cv=7)

In [47]:
print(scores.mean())

0.914285714286


In [48]:
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features=6, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=6, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [50]:
with open('../seedclassifier.pickle', 'wb') as fl:
    pickle.dump(rf, fl)